<a href="https://colab.research.google.com/github/kaedeaki/Data_Science_Internship2023_Data_Glacier/blob/main/Week6_File_ingestion_and_schema_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Week6 File ingestion and schema validation**

##**BIRDS 525 SPECIES- IMAGE CLASSIFICATION (csv)**  
https://www.kaggle.com/datasets/gpiosenka/100-bird-species  





In [1]:
! pip install "git+https://github.com/h2oai/datatable.git"

  Cloning https://github.com/h2oai/datatable.git to /tmp/pip-req-build-9z7zbw3p
  Running command git clone --filter=blob:none --quiet https://github.com/h2oai/datatable.git /tmp/pip-req-build-9z7zbw3p
  Resolved https://github.com/h2oai/datatable.git to commit cb07f6d3569b4646130dbc69041cc0fde95e40fa
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for datatable: filename=datatable-1.1.0a0+build.1691622952.root-cp310-cp310-linux_x86_64.whl size=81735018 sha256=deac8608f9cfc5f06b5d306bdac24e4425ad80c8396fbc6eb7cec2d8749ef7a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tr0kk66r/wheels/30/40/fb/ddceceb6d9bcd86315538e0195412bea90611f182b6ca23cb2
Successfully built datatable


In [2]:
!pip install pip --upgrade

! pip install numpy
! pip install cython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
! pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=0b9be3ac82b0d415f32c65b44460163ef690bfadae3e30e3b2a006f590cfb3c1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
! pip install datatable

In [5]:
! pip install --upgrade pandas
! pip install --upgrade dask

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/e3/59/35a2892bf09ded9c1bf3804461efe772836a5261ef5dfb4e264ce813ff99/pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
  Obtaining dependency information for dask from https://files.pythonhosted.org/packages/bc/07/af1299f8cff6f25cb406421eeb0948a4fbe7de1a49e23dfcd1a5b147919d/dask-2023.8.0-py3-none-any

In [6]:
! pip install ray

  Obtaining dependency information for ray from https://files.pythonhosted.org/packages/b1/5d/292c5fd525b2b762c63659bd8f7bef19eeb44fb6e03a2f9a06aac495b84d/ray-2.6.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.5 MB/s eta 0:00:00


In [7]:

import seaborn as sns
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import dask.dataframe as dd
from pyspark.sql import SparkSession
import multiprocessing as mp
import csv
import datatable as dt
import yaml
import gzip
import ray
import time
from subprocess import check_call
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Reading file with **Pandas**

In [9]:
#importing dataset

start = time.time()

df = pd.read_csv("/content/drive/MyDrive/Data_Glacier/Week6/birds.csv")
df.head()

print(time.time() - start, ' seconds')
display(df)
display(df.info())

3.689972400665283  seconds


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
89880,524.0,valid/BLACK BREASTED PUFFBIRD/3.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89881,524.0,valid/BLACK BREASTED PUFFBIRD/4.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89882,524.0,valid/BLACK BREASTED PUFFBIRD/1.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89883,524.0,valid/BLACK BREASTED PUFFBIRD/2.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89885 entries, 0 to 89884
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   class id         89885 non-null  float64
 1   filepaths        89885 non-null  object 
 2   labels           89885 non-null  object 
 3   data set         89885 non-null  object 
 4   scientific name  89885 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.4+ MB


None

##Reading file with **Pandas with chunk**

In [10]:
start = time.time()

chunks = pd.read_csv("/content/drive/MyDrive/Data_Glacier/Week6/birds.csv", chunksize=50)
df_pandas_chunks = pd.concat(chunks)

print(time.time() - start, ' seconds')
display(df)


1.765298843383789  seconds


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
89880,524.0,valid/BLACK BREASTED PUFFBIRD/3.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89881,524.0,valid/BLACK BREASTED PUFFBIRD/4.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89882,524.0,valid/BLACK BREASTED PUFFBIRD/1.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89883,524.0,valid/BLACK BREASTED PUFFBIRD/2.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS


##Reading file with **Dask**

In [11]:
# Dataframes implement the pandas API

start = time.time()

df = dd.read_csv('/content/drive/MyDrive/Data_Glacier/Week6/birds.csv')
print(time.time() - start, ' seconds')
df.head()

0.012664079666137695  seconds


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


##Reading file with **Dask with chunks**

In [12]:
start = time.time()
df = dd.read_csv('/content/drive/MyDrive/Data_Glacier/Week6/birds.csv', blocksize=25e6)  # 25MB chunks
print(time.time() - start, ' seconds')
df.head()

0.012331008911132812  seconds


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


##Reading file with **Ray**

In [13]:

start = time.time()
ray.data.read_csv('/content/drive/MyDrive/Data_Glacier/Week6/birds.csv')
print(time.time() - start, ' seconds')
df.head()

2023-08-09 23:24:45,013	INFO worker.py:1621 -- Started a local Ray instance.
2023-08-09 23:24:49,199	INFO read_api.py:374 -- To satisfy the requested parallelism of 6, each read task output will be split into 6 smaller blocks.


9.272958278656006  seconds


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


##Basic Validation

In [14]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 5 entries, class id to scientific name
dtypes: object(4), float64(1)

In [15]:
df['class_id'] = df['class id']
df['data_set'] = df['data set']
df['scientific_name'] = df['scientific name']

In [16]:
dtypes = {'class_id': 'object'}

In [17]:
(df.isnull().sum()/df.shape[0])*100

Dask Series Structure:
npartitions=1
class id           object
scientific_name       ...
dtype: object
Dask Name: mul, 12 graph layers

In [18]:
df.head()

,class id,filepaths,labels,data set,scientific name,class_id,data_set,scientific_name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI,0.0,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI,0.0,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI,0.0,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI,0.0,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI,0.0,train,MALACOCINCLA ABBOTTI


##Generate testutility.py

In [19]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


##Generate YAML file

In [20]:
%%writefile file.yaml
file_type: csv
dataset_name: file
file_name: birds
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - class_id
    - filepaths
    - labels
    - data_set
    - scientific_name

Writing file.yaml


In [21]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [22]:
config_data['file_type']

'csv'

In [23]:
config_data['inbound_delimiter']

','

In [24]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'birds',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['class_id', 'filepaths', 'labels', 'data_set', 'scientific_name']}

In [25]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("/content/drive/MyDrive/Data_Glacier/Week6/birds.csv",delimiter=',')
df_sample.head()

,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


In [26]:
current_directory = os.getcwd()
print("current directory:", current_directory)

current directory: /content


In [27]:
# Sets the directory path to direct to the specified location
target_directory = "/content/drive/MyDrive/Data_Glacier/Week6/"

# Change the current directory to the specified location
os.chdir(target_directory)

# Get and display the current directory after change
current_directory = os.getcwd()
print("current directory:", current_directory)

current directory: /content/drive/MyDrive/Data_Glacier/Week6


In [28]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
print("",source_file)

#df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df = pd.read_csv(source_file, delimiter=config_data['inbound_delimiter'])

df.head()

 ./birds.csv


,class id,filepaths,labels,data set,scientific name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


##Validate number of columns and column name of ingested file with YAML.

In [29]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [30]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['class_id', 'filepaths', 'labels', 'data_set', 'scientific_name'], dtype='object')
columns of YAML are: ['class_id', 'filepaths', 'labels', 'data_set', 'scientific_name']


In [31]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")

else:
    print("col validation passed")


column name and column length validation passed
col validation passed


In [32]:
df.head()

,class_id,filepaths,labels,data_set,scientific_name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI


##Write the file in pipe separated text file (|) in gz format.  

In [33]:
output_birds_path = '/content/drive/MyDrive/Data_Glacier/Week6/output_file.csv.gz'

df.to_csv(output_birds_path, sep='|', index=False, compression='gzip')

print(f"File '{output_birds_path}' has been written in gzipped pipe-separated format.")

File '/content/drive/MyDrive/Data_Glacier/Week6/output_file.csv.gz' has been written in gzipped pipe-separated format.


In [34]:
df_gz = pd.read_csv(output_birds_path, sep='|')
df_gz

,class_id,filepaths,labels,data_set,scientific_name
0,0.0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
1,0.0,train/ABBOTTS BABBLER/007.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
2,0.0,train/ABBOTTS BABBLER/008.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
3,0.0,train/ABBOTTS BABBLER/009.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
4,0.0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,train,MALACOCINCLA ABBOTTI
...,...,...,...,...,...
89880,524.0,valid/BLACK BREASTED PUFFBIRD/3.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89881,524.0,valid/BLACK BREASTED PUFFBIRD/4.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89882,524.0,valid/BLACK BREASTED PUFFBIRD/1.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS
89883,524.0,valid/BLACK BREASTED PUFFBIRD/2.jpg,BLACK BREASTED PUFFBIRD,valid,NOTHARCHUS PECTORALIS


In [35]:
csv = pd.read_csv('output_file.csv.gz')

row_count, column_count = df.shape

print("rows:", row_count)
print("columns:", column_count)

rows: 89885
columns: 5


In [36]:
csv = pd.read_csv('output_file.csv.gz')
print(csv)

      class_id|filepaths|labels|data_set|scientific_name
0      0.0|train/ABBOTTS BABBLER/001.jpg|ABBOTTS BABB...
1      0.0|train/ABBOTTS BABBLER/007.jpg|ABBOTTS BABB...
2      0.0|train/ABBOTTS BABBLER/008.jpg|ABBOTTS BABB...
3      0.0|train/ABBOTTS BABBLER/009.jpg|ABBOTTS BABB...
4      0.0|train/ABBOTTS BABBLER/002.jpg|ABBOTTS BABB...
...                                                  ...
89880  524.0|valid/BLACK BREASTED PUFFBIRD/3.jpg|BLAC...
89881  524.0|valid/BLACK BREASTED PUFFBIRD/4.jpg|BLAC...
89882  524.0|valid/BLACK BREASTED PUFFBIRD/1.jpg|BLAC...
89883  524.0|valid/BLACK BREASTED PUFFBIRD/2.jpg|BLAC...
89884  524.0|valid/BLACK BREASTED PUFFBIRD/5.jpg|BLAC...

[89885 rows x 1 columns]


##Summary of the file

In [37]:
import os
import gzip
import csv
from io import StringIO

file_path = 'output_file.csv.gz'
file_size = os.path.getsize(file_path)


with gzip.open(file_path, 'rt', encoding='utf-8') as fOpen:

    data = fOpen.read()
    data_io = StringIO(data)
    reader = csv.reader(data_io)


    header = next(reader, None)
    if header:
        column_count = len(header)


    row_count = sum(1 for _ in data_io)

print("total number of rows:", row_count)
print("total number of columns:", column_count)
print("file size:", file_size, "bytes")


total number of rows: 89885
total number of columns: 1
file size: 296303 bytes
